#### 3회 기출복원 

#### 작업1유형

1. 2022년 데이터 중 2022년 중앙값보다 큰 값의 데이터 수는

In [ ]:
import pandas as pd
import numpy as np
data = pd.read_csv('./t1-data2.csv',index_col='year')
df = data.copy()
df.head()

In [ ]:
df = df.T

In [ ]:
median_2022 = df['2022년'].median()
median_2022

In [ ]:
temp = df[df['2022년']>=median_2022]
temp_2022 = temp['2022년']
result = temp.shape[0]
print(result)

2. 결측치 데이터(행)을 제거하고 앞에서부터 60% 데이터만 활용해 f1칼럼 3사분위 값을 구하시오.(소수점은 절사)

In [ ]:
data = pd.read_csv('./t1-data1.csv')
df = data.copy()
df.head()

df = df.dropna()
df.isna().sum()
print(df.shape)
print(61*0.6)

temp = df.iloc[0:36]
temp.shape
result = temp['f1'].quantile(0.75)
print(result)

3. 결측치가 가장 큰 컬럼명을 구하시오

In [ ]:
data = pd.read_csv('./t1-data1.csv')
df = data.copy()
df.head()

In [ ]:
result = df.isna().sum().sort_values(ascending=False)
print(result.index[0])

#### 작업2유형

- 여행 보험 패키지 상품을 구매할 확률 값을 구하시오
- 평가 지표 : roc-auc

In [1]:
import pandas as pd

In [2]:
train = pd.read_csv('./t2-1-train.csv')
test = pd.read_csv('./t2-1-test.csv')

In [3]:
print(train.shape)
print('===============================================================================')
print(train.info())
print('===============================================================================')
print(test.shape)
print('===============================================================================')
print(test.info())
print('===============================================================================')

(1490, 10)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1490 entries, 0 to 1489
Data columns (total 10 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   id                   1490 non-null   int64  
 1   Age                  1490 non-null   int64  
 2   Employment Type      1490 non-null   object 
 3   GraduateOrNot        1490 non-null   object 
 4   AnnualIncome         1486 non-null   float64
 5   FamilyMembers        1490 non-null   int64  
 6   ChronicDiseases      1490 non-null   int64  
 7   FrequentFlyer        1490 non-null   object 
 8   EverTravelledAbroad  1490 non-null   object 
 9   TravelInsurance      1490 non-null   int64  
dtypes: float64(1), int64(5), object(4)
memory usage: 116.5+ KB
None
(497, 9)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 497 entries, 0 to 496
Data columns (total 9 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  


In [4]:
train.head()

,id,Age,Employment Type,GraduateOrNot,AnnualIncome,FamilyMembers,ChronicDiseases,FrequentFlyer,EverTravelledAbroad,TravelInsurance
0,10000,28,Private Sector/Self Employed,Yes,1250000.0,6,1,No,No,0
1,10001,31,Private Sector/Self Employed,Yes,1250000.0,7,1,No,No,0
2,10002,29,Private Sector/Self Employed,Yes,1200000.0,7,0,No,No,1
3,10003,33,Government Sector,Yes,650000.0,6,1,No,No,1
4,10004,28,Private Sector/Self Employed,Yes,800000.0,6,0,No,Yes,1


In [5]:
train['AnnualIncome'] = train['AnnualIncome'].fillna(train['AnnualIncome'].mean())
test['AnnualIncome'] = test['AnnualIncome'].fillna(test['AnnualIncome'].mean())

In [6]:
target = train.pop('TravelInsurance')

In [7]:
df = pd.concat([train,test])
df.shape

(1987, 9)

In [11]:
from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()
df['Employment Type'] = encoder.fit_transform(df['Employment Type'])
df['GraduateOrNot'] = encoder.fit_transform(df['GraduateOrNot'])
df['FrequentFlyer'] = encoder.fit_transform(df['FrequentFlyer'])
df['EverTravelledAbroad'] = encoder.fit_transform(df['EverTravelledAbroad'])

In [14]:
train = df[:train.shape[0]].copy()
test = df[train.shape[0]:].copy()

In [15]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()

train['AnnualIncome'] = scaler.fit_transform(train[['AnnualIncome']])
test['AnnualIncome'] = scaler.fit_transform(test[['AnnualIncome']])

In [16]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(train,target,test_size=0.2,random_state=2022)
x_train.shape,x_test.shape,y_train.shape,y_test.shape

((1192, 9), (298, 9), (1192,), (298,))

In [35]:
from sklearn.ensemble import RandomForestClassifier
model = RandomForestClassifier(n_estimators=100,max_depth=5)
model.fit(x_train,y_train)
pred = model.predict_proba(x_test)

from sklearn.metrics import roc_auc_score
roc_auc_score(y_test,pred[:,1])

0.7671464646464646

In [36]:
from xgboost import XGBClassifier
model2 = XGBClassifier(n_estimators=100,max_depth=5)
model2.fit(x_train,y_train)
pred2 = model2.predict_proba(x_test)

roc_auc_score(y_test,pred2[:,1])

0.7735353535353535

In [32]:
model = RandomForestClassifier()
model.fit(x_train,y_train)
pred = model.predict_proba(test)

submit = pd.DataFrame()
submit['id'] = test['id']
submit['TravelInsurance'] = pred[:,1]
submit.head()
submit.to_csv("2022.csv", index=False)